In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import time
import sklearn
from tqdm import tqdm

In [2]:
def my_processing(dataset, outsize, pre_type='his'):
    
    data = []
    
    for timestep in tqdm(dataset):                                        #image to crop
        seg = []
        for img in timestep:

            rect_mask = np.zeros(img.shape[:2], dtype="uint8")           #black mask with shape of spectrogram 
            cv2.rectangle(rect_mask,(79,57),(576,427),255,-1)              #fill area we want to keep with white

            masked_img = cv2.bitwise_and(img, img, mask=rect_mask)   #apply


            output_img = masked_img[57:427,79:576]                    #resize image to fit in model
            output_img = cv2.resize(output_img, (outsize,outsize))

            if pre_type == 'his':
                output_img = pre_his(output_img)
                
            output_img = cv2.normalize(output_img, None, 0, 1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
            seg.append(output_img)
            
        data.append(seg)   
    
    return np.array(data)


def pre_his(img):
    yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    y, u, v = cv2.split(yuv)
    y = cv2.equalizeHist(y)
    yuv = cv2.merge([y, u, v])
    his_img = cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)
    return his_img

In [3]:
label_th = 0.1
seg_time = 30
seg_folder = f'/NAS/Benson/Sleep_Apnea/Sleep_Codes/Code_with_data/CNN_LSTM/Model_Training/Segments_{seg_time}s'
info_csv = pd.read_csv(f'label_{seg_time}s_{int(label_th*100)}.csv')
train_data = []
train_label = []
ids = ["00000711-100839", "00000781-100816", "00001096-100779", "00000782-100816"]

for nid in range(len(ids)):
    for x in range(nid*(11700//seg_time), (nid+1)*(11700//seg_time)-9):
        seg_data = []
        for y in range(x, x+10):
            image = cv2.imread(f'{seg_folder}/{info_csv["filename"][y]}.png')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            seg_data.append(image)
        train_data.append(seg_data)
        train_label.append(info_csv["label"][x+9])
        
print(len(train_data))
print(len(train_label))
del info_csv

FileNotFoundError: [Errno 2] No such file or directory: 'label_25s_10.csv'

In [ ]:
# rect_mask = np.zeros(train_data[0].shape[:2], dtype="uint8")           #black mask with shape of spectrogram 
# cv2.rectangle(rect_mask,(79,57),(576,427),255,-1)              #fill area that we want to keep with white

# masked_image = cv2.bitwise_and(train_data[0], train_data[0], mask=rect_mask)   #apply
# output_image = masked_image[57:427,79:576]                     #resize image to fit in model
# output_image = cv2.resize(output_image, (456,456))
# plt.imshow(output_image)

In [ ]:
train_data = my_processing(dataset = train_data, outsize=448, pre_type="his")
train_data, train_label = sklearn.utils.shuffle(train_data, train_label)

X_train = np.array(np.concatenate((train_data[:300], train_data[720:]), axis=0))
Y_train = np.array(np.eye(2)[np.concatenate((train_label[:300], train_label[720:]), axis=0)])
X_test = np.array(train_data[300:540])
Y_test = np.array(np.eye(2)[train_label[300:540]])
X_valid = np.array(train_data[540:720])
Y_valid = np.array(np.eye(2)[train_label[540:720]])

# print(X_train.shape)
# print(Y_train.shape)
# print(X_test.shape)
# print(Y_test.shape)
# print(X_valid.shape)
# print(Y_valid.shape)
del train_data
del train_label

In [ ]:
plt.figure(figsize=(20,10))
for i in range(10):
    ax = plt.subplot(2, 5, i+1)                                    
    plt.title("class : " + str(Y_train[i]), 
               fontsize=18)                                        
    plt.xticks([])                                                
    ax.imshow(X_train[0][i])    

In [ ]:
# import tensorflow as tf
# from tensorflow.core.protobuf import rewriter_config_pb2
# from tensorflow.keras.backend import set_session
# tf.keras.backend.clear_session()  # For easy reset of notebook state.

# config_proto = tf.ConfigProto()
# off = rewriter_config_pb2.RewriterConfig.OFF
# config_proto.graph_options.rewrite_options.arithmetic_optimization = off
# session = tf.Session(config=config_proto)
# set_session(session)

In [ ]:
from tensorflow.keras.layers import TimeDistributed, LSTM, Dense, Conv2D, MaxPool2D, Flatten, Dropout, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model

In [ ]:
def my_model(timesteps, img_height, img_width, img_color_channels, num_classes):
    
    input_batch = Input(shape = (timesteps, img_height, img_width, img_color_channels))
    
    
    cnn_network = TimeDistributed(Conv2D(4, (3,3), padding='same', activation='relu'))(input_batch)
    cnn_network = TimeDistributed(Conv2D(4, (3,3), padding='same', activation='relu'))(cnn_network)
    cnn_network = TimeDistributed(BatchNormalization())(cnn_network)
    cnn_network = TimeDistributed(MaxPool2D(pool_size=(2,2), padding='same', strides=(4,4)))(cnn_network)
    
    cnn_network = TimeDistributed(Conv2D(8, (3,3), padding='same', activation='relu'))(cnn_network)
    cnn_network = TimeDistributed(Conv2D(8, (3,3), padding='same', activation='relu'))(cnn_network)
    cnn_network = TimeDistributed(BatchNormalization())(cnn_network)
    cnn_network = TimeDistributed(MaxPool2D(pool_size=(2,2), padding='same', strides=(4,4)))(cnn_network)
    
    cnn_network = TimeDistributed(Conv2D(8, (3,3), padding='same', activation='relu'))(cnn_network)
    cnn_network = TimeDistributed(Conv2D(8, (3,3), padding='same', activation='relu'))(cnn_network)
    cnn_network = TimeDistributed(BatchNormalization())(cnn_network)
    cnn_network = TimeDistributed(MaxPool2D(pool_size=(2,2), padding='same', strides=(4,4)))(cnn_network)
    
    cnn_network = TimeDistributed(Conv2D(16, (3,3), padding='same', activation='relu'))(cnn_network)
    cnn_network = TimeDistributed(Conv2D(16, (3,3), padding='same', activation='relu'))(cnn_network)
    cnn_network = TimeDistributed(BatchNormalization())(cnn_network)
    cnn_network = TimeDistributed(MaxPool2D(pool_size=(2,2), padding='same', strides=(4,4)))(cnn_network)
    
    cnn_network = TimeDistributed(Flatten())(cnn_network)
   
    
    lstm_network = LSTM(12, return_sequences=False, dropout=0.05, recurrent_dropout=0.05)(cnn_network)
    lstm_network = Dense(4,activation='relu')(lstm_network)
    lstm_network = Dropout(0.1)(lstm_network)
    lstm_network = Dense(2,activation='relu')(lstm_network)
    lstm_network = Dropout(0.1)(lstm_network)
    lstm_network = Dense(num_classes, activation='softmax')(lstm_network)
    
    
    full_network = Model(input_batch, lstm_network)
    return full_network

In [ ]:
model = my_model(timesteps = 10,
                 img_height = 448,
                 img_width = 448,
                 img_color_channels = 3,
                 num_classes = 2)
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

batch_sizes = 10
num_epoch = 1200
start_time = time.perf_counter()
model_history = model.fit(x = X_train,
                    y = Y_train, 
                    batch_size = batch_sizes,
                    epochs = num_epoch,
                    validation_data = (X_valid, Y_valid),
                    validation_batch_size = batch_sizes,
                    callbacks = [EarlyStopping(monitor = 'val_loss',
                                               patience = 60,
                                               verbose = 1)]
                   )
print(f'Total time: {time.perf_counter() - start_time} sec')

In [ ]:
training_loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

plt.plot(training_loss, label="training_loss")
plt.plot(val_loss, label="validation_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Learning Curve")
plt.legend(loc='best')
plt.show()

In [ ]:
plt.plot(model_history.history['accuracy'])
plt.plot(model_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
test_pred = model.evaluate(X_test, Y_test, batch_size=10)
print("test loss, test acc:", test_pred)

In [ ]:
pred = model.predict(X_test, batch_size=10)
print("predictions: ")
print(pred)
print("answer : ")
print(Y_test)

In [ ]:
import sklearn.metrics

print(f'ROC_AUC_Score: {sklearn.metrics.roc_auc_score(Y_test[:,1], pred[:,1])}')
print()
fpr, tpr, _ = sklearn.metrics.roc_curve(Y_test[:,1], pred[:,1])
plt.plot(fpr, tpr)
plt.title('ROC_Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

In [ ]:
pred = pred.argmax(-1)
print('Confusion Matrix: ')
print(sklearn.metrics.confusion_matrix(Y_test.argmax(-1), pred))
print()
print(f'Cohen_Kappa_Score: {sklearn.metrics.cohen_kappa_score(Y_test.argmax(-1), pred)}')

In [ ]:
# model.save("lstm.h5(1)")